In [2]:
import os
import pandas as pd
from Bio import SeqIO
import yaml
import numpy as np

# load in config with GPSC paths
with open("config/TB_reps.yaml", 'r') as file:
    config=yaml.safe_load(file)

# extract GPSCs
gpscs = config['samples'] 

# set params
amplicon_stats = list()
xlen = 2200

total_genome_coverages={}
amplicon_positions={}

with open('output.txt', 'w') as f:

    for gpsc, fasta_file in gpscs.items():
        records = list(SeqIO.parse(fasta_file, "fasta"))

        # calculate length of the genome for each GPSC
        genome_length=sum(len(record.seq) for record in records)

        # initialize amplicon genome coverage for each GPSC separately 
        total_genome_coverage=0

        # initialize set of covered positions across each GPSC 
        covered_positions = set()

        # intitialize list of amplicon positions 
        amplicon_positions[gpsc] = []

        print(f"Processing {gpsc}...", file=f)  

        # load the samtools depth file
        fwd_depth_file = os.path.join("samtools_depth_indiv_primers", f"{gpsc}_fwd.depth")
        rev_depth_file = os.path.join("samtools_depth_indiv_primers", f"{gpsc}_rev.depth")
        
        # load each depth file into a df
        fwd_df = pd.read_csv(fwd_depth_file, sep="\t", names=["Ref", "Pos", "Depth"])
        fwd_df['Primer'] = 'fwd'

        rev_df = pd.read_csv(rev_depth_file, sep="\t", names=["Ref", "Pos", "Depth"])
        rev_df['Primer'] = 'rev'

        # combine the dfs
        df = pd.concat([fwd_df, rev_df])

        # filter for fwd and rev positions separately 
        fwd_primer_binding_sites = df[(df["Depth"] == 1) & (df["Primer"] == 'fwd')]["Pos"].tolist()

        rev_primer_binding_sites = df[(df["Depth"] == 1) & (df["Primer"] == 'rev')]["Pos"].tolist()

        def calculate_amplicons(primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, primer_direction):
            for p1loc in primer_binding_sites:
        # find the next primer binding site within xlen bases
                p2loc = next((pos for pos in primer_binding_sites if p1loc < pos <= p1loc + xlen), None)

                if p2loc is not None:
            # amp stats
                    amplicon_stats.append((gpsc, gpsc, gpsc, gpsc, gpsc, gpsc, 0, 0, 0, p1loc, p2loc, primer_direction))
                    covered_positions.update(range(p1loc, p2loc+1))
            # ID amp positions
                    amplicon_positions[gpsc].append((p1loc, p2loc))
                    print(f"Detected amplicon from {p1loc} to {p2loc}.", file=f)  

            # get total genome coverage
                    total_genome_coverage += p2loc - p1loc

        # calculate predicted % coverage 
            coverage_percentage = (len(covered_positions) / genome_length) * 100

        # update dictionary 
            total_genome_coverages[gpsc] = coverage_percentage

        # summarize fwd amplicons
        calculate_amplicons(fwd_primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, 'fwd')
        # summarize rev amplicons
        calculate_amplicons(rev_primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, 'rev')

colnames = ["pid1", "pid2", "set1", "set2",
            "pseq1", "pseq2",
            "max_hdist", "hdist1", "hdist2",
            "p1loc", "p2loc", "PrimerDirection"]
coltypes = ["<U30", "<U30", "<U30", "<U30",
            "<U30", "<U30",
            float, float, float,
            int, int, "<U3"]

dt = {'names': colnames, 'formats': coltypes}

amplicon_statsnp = np.array(amplicon_stats,
                     dtype=dt)

np.save("amplicon_statstab.npy", amplicon_statsnp)

FileNotFoundError: [Errno 2] No such file or directory: 'samtools_depth_indiv_primers/S.agalactiae_fwd.depth'

In [3]:
import csv

# Initialize a dictionary to group amplicons by sequence and primer type
grouped_amplicons = {}

# Define the maximum allowed size for amplicons
max_amplicon_size = 2000

# Iterate over amplicon_stats, assuming the last element is primer_direction, and the first is gpsc
for amplicon in amplicon_stats:
    *_, p1loc, p2loc, primer_direction = amplicon
    gpsc = amplicon[0]
    key = (gpsc, primer_direction)
    if key not in grouped_amplicons:
        grouped_amplicons[key] = []
    grouped_amplicons[key].append((p1loc, p2loc))

# Initialize list for contiguous amplicons
contiguous_amplicons = []

# Process each group for contiguity
for (gpsc, primer_direction), positions in grouped_amplicons.items():
    positions.sort()
    start, end = positions[0]
    
    for current_start, current_end in positions[1:]:
        # Check if contiguous and does not exceed max amplicon size
        if current_start <= end + 1 and (current_end - start) <= max_amplicon_size:
            end = max(end, current_end)
        else:
            contiguous_amplicons.append([gpsc, start, end, primer_direction])
            start, end = current_start, current_end
    # After processing all positions, append the last contiguous segment
    contiguous_amplicons.append([gpsc, start, end, primer_direction])

# Write contiguous amplicons to CSV
with open('amplicon_positions.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Sequence", "Start", "End", "Primer Type"])
    for amplicon in contiguous_amplicons:
        csvwriter.writerow(amplicon)

In [ ]:
            ------------------------------------------------------ separate fwd and rev amplicon predictions --------------------------------------------------------------

In [6]:
import pandas as pd

# create csv file containing predicted genome coverage for each sequence based on predicted amplicon coverage
total_genome_coverages_df = pd.DataFrame([total_genome_coverages])

total_genome_coverages_df.to_csv('genome_coverage_pc.csv', index=False)

In [7]:
for gpsc, coverage in total_genome_coverages.items():
    print(f"{gpsc}: {coverage}%")

H.influenzae: 0.003879488869145387%
H37Rv: 94.29513375398841%
K.pneumoniae: 0.03167923478560163%
M.abscessus: 0.39467774135158623%
M.canetti: 89.43826698968014%
M.fortuitum: 0.5697250983129756%
M.intracellulare: 2.1540788708430068%
OXC141: 0.005449545797541027%
PA01: 0.03133578230267396%


In [8]:
from Bio import SeqIO

# Load the config file
with open("config/TB_reps.yaml", 'r') as file:
    config=yaml.safe_load(file)

# Extract the GPSCs
gpscs = config['samples']

# Initialize a dictionary to store the genome lengths
genome_lengths = {}

# Calculate and store the length of each sequence
for gpsc, fasta_file in gpscs.items():
    records = list(SeqIO.parse(fasta_file, "fasta"))
    genome_length = sum(len(record.seq) for record in records)
    genome_lengths[gpsc] = genome_length  # Store the genome length in the dictionary

#Print the genome lengths
for gpsc, length in genome_lengths.items():
    print(f"{gpsc}: {length} bp")

H.influenzae: 1830138 bp
H37Rv: 4411532 bp
K.pneumoniae: 5438894 bp
M.abscessus: 5067172 bp
M.canetti: 4432426 bp
M.fortuitum: 6406072 bp
M.intracellulare: 5402402 bp
OXC141: 2036867 bp
PA01: 6264404 bp
S.aureus: 2872762 bp
S.odontolytica: 2455831 bp


In [9]:
import csv
import yaml
from Bio import SeqIO

# Load the config file
with open("config/TB_reps.yaml", 'r') as file:
    config = yaml.safe_load(file)

# Extract the GPSCs
gpscs = config['samples']

# Open a CSV file to write
with open('assembly_lengths.csv', mode='w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    
    # Calculate and store the length of each sequence, then write to CSV
    for gpsc, fasta_file in gpscs.items():
        records = list(SeqIO.parse(fasta_file, "fasta"))
        genome_length = sum(len(record.seq) for record in records)
        # Write the ID, start (0), and the length to the CSV file
        csv_writer.writerow([gpsc, 0, genome_length])